In [1]:
import os
import sys
from pathlib import Path

from datasets import Dataset
from langchain_community.document_loaders import (
    DirectoryLoader,
    UnstructuredMarkdownLoader,
)
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.llms import Ollama
from ragas import evaluate
from ragas.metrics import answer_correctness, faithfulness
from ragas.testset.evolutions import multi_context, reasoning, simple
from ragas.testset.generator import TestsetGenerator

sys.path.append(str(Path.cwd().parents[1]))  # nopep8
from utilities.ollama import verify_ollama_model_present
from utilities.project_paths import sample_document_dirpath

MODEL_NAME = "openchat"
# https://ollama.com/library/openchat

generator_llm = Ollama(model=MODEL_NAME)
embeddings = OllamaEmbeddings(model=MODEL_NAME)

# make sure that the ollama is available to use locally (might need to ollama pull)
verify_ollama_model_present("openchat")


d:\projects\testbed\ragas_cookbook\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


validating model exists...


In [8]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader, TextLoader
from langchain_text_splitters import MarkdownHeaderTextSplitter

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

loader = DirectoryLoader(
    path=sample_document_dirpath,
    loader_cls=UnstructuredMarkdownLoader,
    show_progress=True
)
raw_documents = loader.load()

markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on, strip_headers=False
)

# split it into chunks
docs = []
for i in raw_documents:
    docs.extend(markdown_splitter.split_text(i.page_content))


generator = TestsetGenerator.from_langchain(
    generator_llm=generator_llm,
    critic_llm=generator_llm,
    embeddings=embeddings
)

100%|██████████| 11/11 [00:00<00:00, 14.10it/s]


In [7]:
# generate testset
testset = generator.generate_with_langchain_docs(documents,
                                                 test_size=len(documents), distributions={
                                                     simple: 0.5, reasoning: 0.3, multi_context: 0.2})

df = testset.to_pandas()
df.to_excel('synthetic_testset.xlsx')
print(df)

KeyboardInterrupt: 

In [6]:
documents

[Document(page_content='AWS Chalice and AWS Lambda REST API example\n\nPurpose\n\nShows how to use AWS Chalice with the AWS SDK for Python (Boto3) to \ncreate a serverless REST API that uses Amazon API Gateway, AWS Lambda, and \nAmazon DynamoDB. The REST API simulates a system that tracks daily cases\nof COVID-19 in the United States, using fictional data. Learn how to:\n\nUse AWS Chalice to define routes in AWS Lambda functions that\n are called to handle REST requests that come through Amazon API Gateway.\n\nUse AWS Lambda functions to retrieve and store data in an Amazon DynamoDB \ntable to serve REST requests.\n\nDefine table structure and security role resources in an AWS CloudFormation template.\n\nUse AWS Chalice and AWS CloudFormation to package and deploy all necessary resources.\n\nUse AWS CloudFormation to clean up all created resources.\n\nThis example brings together some of the same information you can find in the\ntutorials in the \nAWS Chalice GitHub repository.\n\nPrer